In [1]:
from collections import Counter

In [2]:
def isGoal(state,goalState):
    if state == goalState:
        return True
    return False

In [3]:
def moveZero(state,newZeroIndex):
    state=list(state)
    i=state.index('0')
    state[i],state[newZeroIndex]=state[newZeroIndex],state[i]
   # state=str(state)
    return state

In [4]:
def findNeighbours(state):
    neighbours=[]
    zero=state.find('0')
    if zero > 2:
        neighbours.append(moveZero(state,zero-3))
    if zero < 6:
        neighbours.append(moveZero(state,zero+3))
    if zero == 0 or zero == 1 or zero == 3 or zero == 4 or zero == 6 or zero == 7:
        neighbours.append(moveZero(state,zero+1))
    if zero == 1 or zero == 2 or zero == 4 or zero == 5 or zero == 7 or zero == 8:
        neighbours.append(moveZero(state,zero-1))
    return neighbours

In [5]:
def BFS(state,goal):
    frontier={state:True} ## Making it dict "or map" for optimization
    explored=set()
    parent_map={state:None}
    while frontier:
        currState=next(iter(frontier)) ##getting the first element in the map , same as queue
        frontier.pop(currState)
        explored.add(currState) ##Explored set
        if isGoal(currState,goal):
            return parent_map , currState , explored
        for neighbour in findNeighbours(currState):
            string=""
            neighbour=string.join(neighbour) ## neighbour is a list of charachters , so they are converted to one string
            if neighbour not in frontier and neighbour not in explored:
                frontier[neighbour]=True
                parent_map[neighbour]=currState

In [6]:
import math
def manhattanDist(x_start,y_start,x_goal,y_goal):
    return abs(x_start-x_goal)+abs(y_start-y_goal)

def euclideanDist(x_start,y_start,x_goal,y_goal):
    return math.sqrt((x_start-x_goal)**2+(y_start-y_goal)**2)
    

In [7]:
def heuristic(equation,state,goal):
    state=list(state)
    goal=list(goal)
    H=0
    for i,s in enumerate(state):
        if s=='0':
            i=0
            s=0
        else:
            s=goal.index(s)
        x_start,y_start=int(i/3),i%3
        x_goal,y_goal=int(s/3),int(s)%3
        
        if equation=="manhattan":
            H+=manhattanDist(x_start,y_start,x_goal,y_goal)
        elif equation=="euclidean":
            H+=euclideanDist(x_start,y_start,x_goal,y_goal)
            
    return H

In [8]:
def decrease_key(F,equation,currState,G,goal):
    Fnew=heuristic(equation,currState,goal) + G
    return min(Fnew,F)
    

In [9]:
def A_star(equation,state,goal):
    frontier={state: heuristic(equation,state,goal)}
    parent_map={state: None}
    explored=set()
    
    while frontier:
        currState=min(frontier , key = frontier.get)
        G=frontier[currState] - heuristic(equation,currState,goal) + 1 #F=G+H ,G=F-H
        explored.add(currState)
        frontier.pop(currState)
        
        if isGoal(currState,goal):
            return parent_map,currState,explored
        
        
        for neighbour in findNeighbours(currState):
            string=""
            neighbour=string.join(neighbour) ## neighbour is a list of charachters , so they are converted to one string
            if neighbour not in frontier and neighbour not in explored:
                frontier[neighbour]=heuristic(equation,neighbour,goal)+G
                parent_map[neighbour]=currState
            elif neighbour in frontier:
                frontier[neighbour]=decrease_key(frontier[neighbour],equation,neighbour,G,goal)
    return False


In [10]:
def deliverables(parent_map,explored,currState,startState):
    path_to_goal=[]
    cost=0
    current=currState
    while parent_map[current]:
        cost+=1
        path_to_goal.append(current)
        current=parent_map[current]
    path_to_goal.append(startState)
    return path_to_goal , cost

In [11]:
def loopOverState(state):
    countDect = Counter(state)
    for char, count in countDect.items():
        if (count > 1):
            return False
        if (char =='9'):
            return False
    return True

In [12]:
def checkInput(state):
    if len(state)!=9:
        return False
    if (loopOverState(state)==False):
        return False
    if (state.isnumeric()==False):
        return False
    return True

In [13]:
def checkIfSolvable(state):
    inversions=0
    for i in range(0,9):
        for j in range(i+1,9):
            if int(state[i])>int(state[j]) and int(state[i])!=0 and int(state[j])!=0:
                inversions+=1
    if(inversions%2==0):
        return True
    else:
        return False

In [14]:
def printing(printable,goal):
    if(goal=="path"):
        printable.reverse()
    for i in range(len(printable)):
        for j in range(0,10,3):
            print(" ".join(printable[i][j:j+3]))
        print("======================================")

In [15]:
def startGame():
    import time
    startState=input("Enter start state:\n")
    if(checkInput(startState)==False):
        print("Invalid input\n")
    elif(checkIfSolvable(startState)==False):
        print("Puzzle is not solvable\n")
    else:
        parent_map={}
        explored=set()
        currentState=startState
        t0=time.time()
        #parent_map,currentState,explored=BFS(startState,"012345678")
        parent_map,currentState,explored=A_star("manhattan",currentState,"012345678")
        p,c=deliverables(parent_map,explored,currentState,startState)
        t1=time.time()
        time=t1-t0
        print("Path to Goal:\n")
        print("=============\n")
        printing(p,"path")
        print("Explored:\n")
        print("========\n")
        #printing(list(explored),"e")     
        print("Time expanded:\n==============\n")
        print(time)
        print("\nCost:\n====\n")
        print(c)

In [16]:
startGame()

Enter start state:
087654321
Path to Goal:


0 8 7
6 5 4
3 2 1

8 0 7
6 5 4
3 2 1

8 5 7
6 0 4
3 2 1

8 5 7
0 6 4
3 2 1

0 5 7
8 6 4
3 2 1

5 0 7
8 6 4
3 2 1

5 7 0
8 6 4
3 2 1

5 7 4
8 6 0
3 2 1

5 7 4
8 6 1
3 2 0

5 7 4
8 6 1
3 0 2

5 7 4
8 0 1
3 6 2

5 7 4
0 8 1
3 6 2

5 7 4
3 8 1
0 6 2

5 7 4
3 8 1
6 0 2

5 7 4
3 0 1
6 8 2

5 0 4
3 7 1
6 8 2

5 4 0
3 7 1
6 8 2

5 4 1
3 7 0
6 8 2

5 4 1
3 7 2
6 8 0

5 4 1
3 7 2
6 0 8

5 4 1
3 0 2
6 7 8

5 0 1
3 4 2
6 7 8

0 5 1
3 4 2
6 7 8

3 5 1
0 4 2
6 7 8

3 5 1
4 0 2
6 7 8

3 0 1
4 5 2
6 7 8

3 1 0
4 5 2
6 7 8

3 1 2
4 5 0
6 7 8

3 1 2
4 0 5
6 7 8

3 1 2
0 4 5
6 7 8

0 1 2
3 4 5
6 7 8

Explored:


Time expanded:

3.5219523906707764

Cost:
====

30
